In [1]:
import json
import os
import tqdm

from tqdm import tqdm

In [2]:
# load the result_comparison sentences with manuscript and cited paper ids
result_comps = []
with open("../misc/result_comps.jsonl", "r") as f:
    for line in f:
        p = json.loads(line.strip("\n"))
        for k in set(p.keys()): 
            if k.endswith("prob"): del p[k]
        del p["intent"]
        result_comps.append(p)
        
        if len(result_comps) == 100: break
result_comps[0]

{'citation_sentence': 'In addition, we here report comparable changes in serum cholesterol and lipoprotein concentrations as found in previously published meta-analyses [1] [2] [3] , implying that the included studies in this meta-analyses are representative of all available studies in the literature that have been performed with plant sterols and plant stanols.',
 'manuscript_id': '8281087',
 'cited_id': '15228934'}

In [3]:
# get all involved paper ids
manuscript_ids = {p["manuscript_id"]: [] for p in result_comps}
cited_ids = {p["cited_id"]: [] for p in result_comps}

for i, p in enumerate(result_comps):
    manuscript_ids[p["manuscript_id"]].append(i)
    cited_ids[p["cited_id"]].append(i)

In [4]:
# for each paper in s2orc,
# append its text to the result_comparison sentences that are involved with it
with open("/home/jessica/data/s2orc/s2orc_result_subset.jsonl", "r") as f:
    for i, line in tqdm(enumerate(f)):
        p = json.loads(line)
        p_id = p["paper_id"]
        
        if p_id in manuscript_ids: 
            for row_id in manuscript_ids[p_id]:
                result_comps[row_id]["manuscript_text"] = p["body_text"]
        if p_id in cited_ids: 
            for row_id in cited_ids[p_id]:
                result_comps[row_id]["cited_text"] = p["body_text"]

982974it [01:25, 11504.84it/s]


In [5]:
# function to extract result_text from manuscripts
def get_result_text(body_text):
    result_text = []

    i = 0
    is_result = 0
    is_discuss = False 
    while i < len(body_text) and not is_discuss:
        p = body_text[i]
        i += 1
        
        section_name = p["section"].lower()
        is_result += int("result" in section_name or "find" in section_name)
        is_discuss = "discuss" in section_name or "conclu" in section_name

        if is_result and not is_discuss: 
            result_text.append(p)

    return result_text

In [12]:
# replace cited_texts by plaintext
for p in result_comps: 
    cited_text = " ".join(para["text"] for para in p["cited_text"])
    result_text = get_result_text(p["manuscript_text"])
    result_text = " ".join(para["text"] for para in result_text)
    
    p["cited_text"] = cited_text
    p["manuscript_result_text"] = result_text
    
    del p["manuscript_text"]

In [13]:
with open("../misc/test_dataset.jsonl", "w") as outfile:
    for p in result_comps:
        outfile.write(json.dumps(p) + "\n")

In [14]:
rc = []
with open("../misc/test_dataset.jsonl", "r") as f:
    for line in f:
        rc.append(json.loads(line))
print(rc == result_comps)

True
